In [1]:
from functools import reduce
import os
import pandas as pd
import time

Create the lists of Census table columns

In [2]:
race_cols = ['B03002_001E', 'B03002_003E']
age_cols = ['B01001_003E', 'B01001_027E', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E']
working_age_pop_cols = ['B23001_006E','B23001_013E','B23001_020E','B23001_027E','B23001_034E','B23001_041E','B23001_048E','B23001_055E','B23001_062E','B23001_069E','B23001_074E','B23001_079E','B23001_084E','B23001_092E','B23001_099E','B23001_106E','B23001_113E','B23001_120E','B23001_127E','B23001_134E','B23001_141E','B23001_148E','B23001_155E','B23001_160E','B23001_165E','B23001_170E']
unemployed_pop_cols = ['B23001_008E','B23001_015E','B23001_022E','B23001_029E','B23001_036E','B23001_043E','B23001_050E','B23001_057E','B23001_064E','B23001_071E','B23001_076E','B23001_081E','B23001_086E','B23001_094E','B23001_101E','B23001_108E','B23001_115E','B23001_122E','B23001_129E','B23001_136E','B23001_143E','B23001_150E','B23001_157E','B23001_162E','B23001_167E','B23001_172E']
med_hh_inc_cols = ['B19013_001E']
snap_cols = ['B22001_002E', 'B22001_005E']
ownership_cols = ['B25003_002E', 'B25003_003E']

Read in the Census API key

In [3]:
key = open('census_key.txt', 'r').read()

Define the function to make API calls

In [4]:
def get_census_year(year, variables, geography):
    census_endpoint = 'https://api.census.gov/data/'
    census_path = '/acs5' if year == 2009 else '/acs/acs5'
    census_url = census_endpoint + str(year) + census_path + '?get=' + ','.join(variables) + ',NAME&for=' + geography.replace(' ','%20') + ':*&key=' + key
    
    results = pd.read_json(census_url)
    results = results.rename(columns=results.iloc[0])[1:]
    results['year'] = year
    
    time.sleep(2)
    
    return results

def get_census_years(from_year, to_year, variables, geography):
    results = pd.DataFrame() # Create an empty dataframe
    
    for year in range(from_year, to_year):
        results = results.append(get_census_year(year, variables, geography))
    
    return results

## Race, age, median household income, SNAP enrollment, own vs. rent

### Counties and county equivalents

In [5]:
def read_or_save(name, func):
    path = 'pickles/' + name + '.pickle.gz'
    if (os.path.isfile(path)):
        return pd.read_pickle(path)
    else:
        result = func()
        os.makedirs('pickles', exist_ok=True)
        result.to_pickle(path)
        return result

#### Make the API calls

In [6]:
# Concatenate the items of the lists of table columns into a single list
race_age_inc_snap_ownership_cols = race_cols + age_cols + med_hh_inc_cols + snap_cols + ownership_cols

# Iterate through the years
race_age_inc_snap_ownership_counties = read_or_save(
    'race_age_inc_snap_ownership_counties',
    lambda: get_census_years(2009, 2018, race_age_inc_snap_ownership_cols, 'county')
)

#### Format the returned data

In [7]:
# Convert data columns to numeric
race_age_inc_snap_ownership_counties[race_age_inc_snap_ownership_cols] = race_age_inc_snap_ownership_counties[race_age_inc_snap_ownership_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
race_age_inc_snap_ownership_counties['under_age_5'] = race_age_inc_snap_ownership_counties[age_cols[:2]].sum(axis=1)
race_age_inc_snap_ownership_counties['age_65_and_over'] = race_age_inc_snap_ownership_counties[age_cols[2:]].sum(axis=1)
race_age_inc_snap_ownership_counties['non_white'] = race_age_inc_snap_ownership_counties['B03002_001E'] - race_age_inc_snap_ownership_counties['B03002_003E']
race_age_inc_snap_ownership_counties['pct_non_white'] = race_age_inc_snap_ownership_counties['non_white'] / race_age_inc_snap_ownership_counties['B03002_001E']
race_age_inc_snap_ownership_counties['pct_under_age_5'] = race_age_inc_snap_ownership_counties['under_age_5'] / race_age_inc_snap_ownership_counties['B03002_001E']
race_age_inc_snap_ownership_counties['pct_age_65_and_over'] = race_age_inc_snap_ownership_counties['age_65_and_over'] / race_age_inc_snap_ownership_counties['B03002_001E']
race_age_inc_snap_ownership_counties['pct_under_age_5_or_age_65_and_over'] = (race_age_inc_snap_ownership_counties['under_age_5'] + race_age_inc_snap_ownership_counties['age_65_and_over']) / race_age_inc_snap_ownership_counties['B03002_001E']
race_age_inc_snap_ownership_counties['pct_hh_snap'] = race_age_inc_snap_ownership_counties['B22001_002E'] / (race_age_inc_snap_ownership_counties['B22001_002E'] + race_age_inc_snap_ownership_counties['B22001_005E'])
race_age_inc_snap_ownership_counties['pct_hh_renter_occupied'] = race_age_inc_snap_ownership_counties['B25003_003E'] / (race_age_inc_snap_ownership_counties['B25003_002E'] + race_age_inc_snap_ownership_counties['B25003_003E'])
race_age_inc_snap_ownership_counties['fips'] = race_age_inc_snap_ownership_counties['state'] + race_age_inc_snap_ownership_counties['county']

# Drop the columns we no longer need
race_age_inc_snap_ownership_counties = race_age_inc_snap_ownership_counties.drop(age_cols, axis=1)
race_age_inc_snap_ownership_counties = race_age_inc_snap_ownership_counties.drop(['B03002_003E', 'B22001_005E', 'B25003_002E', 'state', 'county'], axis=1)

# Rename and reorder the columns
race_age_inc_snap_ownership_counties.columns = ['total_pop', 'med_hh_inc', 'hh_snap', 'hh_renter_occupied', 'geography', 'year', 'under_age_5', 'age_65_and_over', 'non_white',
                                                'pct_non_white', 'pct_under_age_5', 'pct_age_65_and_over', 'pct_under_age_5_or_age_65_and_over', 'pct_hh_snap', 'pct_hh_renter_occupied',
                                                'fips']
race_age_inc_snap_ownership_counties = race_age_inc_snap_ownership_counties[['year', 'fips', 'geography', 'total_pop', 'non_white', 'pct_non_white',
                                                                             'under_age_5', 'pct_under_age_5', 'age_65_and_over', 'pct_age_65_and_over', 'pct_under_age_5_or_age_65_and_over',
                                                                             'med_hh_inc', 'hh_snap', 'pct_hh_snap', 'hh_renter_occupied', 'pct_hh_renter_occupied']]

race_age_inc_snap_ownership_counties = race_age_inc_snap_ownership_counties
race_age_inc_snap_ownership_counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28985 entries, 1 to 3220
Data columns (total 16 columns):
year                                  28985 non-null int64
fips                                  28985 non-null object
geography                             28985 non-null object
total_pop                             28985 non-null float64
non_white                             28985 non-null float64
pct_non_white                         28985 non-null float64
under_age_5                           28985 non-null float64
pct_under_age_5                       28985 non-null float64
age_65_and_over                       28985 non-null float64
pct_age_65_and_over                   28985 non-null float64
pct_under_age_5_or_age_65_and_over    28985 non-null float64
med_hh_inc                            28984 non-null float64
hh_snap                               28985 non-null float64
pct_hh_snap                           28985 non-null float64
hh_renter_occupied                    2898

### Native American Reservations

#### Make the API calls

In [8]:
# Concatenate the items of the lists of table columns into a single list
race_age_inc_snap_ownership_cols = race_cols + age_cols + med_hh_inc_cols + snap_cols + ownership_cols

# Iterate through the years
race_age_inc_snap_ownership_reservations = read_or_save(
    'race_age_inc_snap_ownership_reservations',
    lambda: get_census_years(2009, 2018, race_age_inc_snap_ownership_cols, 'american indian area/alaska native area/hawaiian home land')
)

#### Format the returned data

In [9]:
# Convert data columns to numeric
race_age_inc_snap_ownership_reservations[race_age_inc_snap_ownership_cols] = race_age_inc_snap_ownership_reservations[race_age_inc_snap_ownership_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
race_age_inc_snap_ownership_reservations['under_age_5'] = race_age_inc_snap_ownership_reservations[age_cols[:2]].sum(axis=1)
race_age_inc_snap_ownership_reservations['age_65_and_over'] = race_age_inc_snap_ownership_reservations[age_cols[2:]].sum(axis=1)
race_age_inc_snap_ownership_reservations['non_white'] = race_age_inc_snap_ownership_reservations['B03002_001E'] - race_age_inc_snap_ownership_reservations['B03002_003E']
race_age_inc_snap_ownership_reservations['pct_non_white'] = race_age_inc_snap_ownership_reservations['non_white'] / race_age_inc_snap_ownership_reservations['B03002_001E']
race_age_inc_snap_ownership_reservations['pct_under_age_5'] = race_age_inc_snap_ownership_reservations['under_age_5'] / race_age_inc_snap_ownership_reservations['B03002_001E']
race_age_inc_snap_ownership_reservations['pct_age_65_and_over'] = race_age_inc_snap_ownership_reservations['age_65_and_over'] / race_age_inc_snap_ownership_reservations['B03002_001E']
race_age_inc_snap_ownership_reservations['pct_under_age_5_or_age_65_and_over'] = (race_age_inc_snap_ownership_reservations['under_age_5'] + race_age_inc_snap_ownership_reservations['age_65_and_over']) / race_age_inc_snap_ownership_reservations['B03002_001E']
race_age_inc_snap_ownership_reservations['pct_hh_snap'] = race_age_inc_snap_ownership_reservations['B22001_002E'] / (race_age_inc_snap_ownership_reservations['B22001_002E'] + race_age_inc_snap_ownership_reservations['B22001_005E'])
race_age_inc_snap_ownership_reservations['pct_hh_renter_occupied'] = race_age_inc_snap_ownership_reservations['B25003_003E'] / (race_age_inc_snap_ownership_reservations['B25003_002E'] + race_age_inc_snap_ownership_reservations['B25003_003E'])

# Drop the columns we no longer need
race_age_inc_snap_ownership_reservations = race_age_inc_snap_ownership_reservations.drop(age_cols, axis=1)
race_age_inc_snap_ownership_reservations = race_age_inc_snap_ownership_reservations.drop(['B03002_003E', 'B22001_005E', 'B25003_002E'], axis=1)

# Rename and reorder the columns
race_age_inc_snap_ownership_reservations.columns = ['total_pop', 'med_hh_inc', 'hh_snap', 'hh_renter_occupied', 'geography', 'fips', 'year', 'under_age_5', 'age_65_and_over', 'non_white',
                                                'pct_non_white', 'pct_under_age_5', 'pct_age_65_and_over', 'pct_under_age_5_or_age_65_and_over', 'pct_hh_snap', 'pct_hh_renter_occupied']
race_age_inc_snap_ownership_reservations = race_age_inc_snap_ownership_reservations[['year', 'fips', 'geography', 'total_pop', 'non_white', 'pct_non_white',
                                                                                     'under_age_5', 'pct_under_age_5', 'age_65_and_over', 'pct_age_65_and_over', 'pct_under_age_5_or_age_65_and_over',
                                                                                     'med_hh_inc', 'hh_snap', 'pct_hh_snap', 'hh_renter_occupied', 'pct_hh_renter_occupied']]

race_age_inc_snap_ownership_reservations = race_age_inc_snap_ownership_reservations
race_age_inc_snap_ownership_reservations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6200 entries, 1 to 694
Data columns (total 16 columns):
year                                  6200 non-null int64
fips                                  6200 non-null object
geography                             6200 non-null object
total_pop                             6200 non-null float64
non_white                             6200 non-null float64
pct_non_white                         5474 non-null float64
under_age_5                           6200 non-null float64
pct_under_age_5                       5474 non-null float64
age_65_and_over                       6200 non-null float64
pct_age_65_and_over                   5474 non-null float64
pct_under_age_5_or_age_65_and_over    5474 non-null float64
med_hh_inc                            6056 non-null float64
hh_snap                               6200 non-null float64
pct_hh_snap                           5454 non-null float64
hh_renter_occupied                    6200 non-null float6

### United States

#### Make the API calls

In [10]:
# Concatenate the items of the lists of table columns into a single list
race_age_inc_snap_ownership_cols = race_cols + age_cols + med_hh_inc_cols + snap_cols + ownership_cols

# Iterate through the years
race_age_inc_snap_ownership_us = read_or_save(
    'race_age_inc_snap_ownership_us',
    lambda: get_census_years(2009, 2018, race_age_inc_snap_ownership_cols, 'us')
)

#### Format the returned data

In [11]:
# Convert data columns to numeric
race_age_inc_snap_ownership_us[race_age_inc_snap_ownership_cols] = race_age_inc_snap_ownership_us[race_age_inc_snap_ownership_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
race_age_inc_snap_ownership_us['under_age_5'] = race_age_inc_snap_ownership_us[age_cols[:2]].sum(axis=1)
race_age_inc_snap_ownership_us['age_65_and_over'] = race_age_inc_snap_ownership_us[age_cols[2:]].sum(axis=1)
race_age_inc_snap_ownership_us['non_white'] = race_age_inc_snap_ownership_us['B03002_001E'] - race_age_inc_snap_ownership_us['B03002_003E']
race_age_inc_snap_ownership_us['pct_non_white'] = race_age_inc_snap_ownership_us['non_white'] / race_age_inc_snap_ownership_us['B03002_001E']
race_age_inc_snap_ownership_us['pct_under_age_5'] = race_age_inc_snap_ownership_us['under_age_5'] / race_age_inc_snap_ownership_us['B03002_001E']
race_age_inc_snap_ownership_us['pct_age_65_and_over'] = race_age_inc_snap_ownership_us['age_65_and_over'] / race_age_inc_snap_ownership_us['B03002_001E']
race_age_inc_snap_ownership_us['pct_under_age_5_or_age_65_and_over'] = (race_age_inc_snap_ownership_us['under_age_5'] + race_age_inc_snap_ownership_us['age_65_and_over']) / race_age_inc_snap_ownership_us['B03002_001E']
race_age_inc_snap_ownership_us['pct_hh_snap'] = race_age_inc_snap_ownership_us['B22001_002E'] / (race_age_inc_snap_ownership_us['B22001_002E'] + race_age_inc_snap_ownership_us['B22001_005E'])
race_age_inc_snap_ownership_us['pct_hh_renter_occupied'] = race_age_inc_snap_ownership_us['B25003_003E'] / (race_age_inc_snap_ownership_us['B25003_002E'] + race_age_inc_snap_ownership_us['B25003_003E'])

# Drop the columns we no longer need
race_age_inc_snap_ownership_us = race_age_inc_snap_ownership_us.drop(age_cols, axis=1)
race_age_inc_snap_ownership_us = race_age_inc_snap_ownership_us.drop(['B03002_003E', 'B22001_005E', 'B25003_002E'], axis=1)

# Rename and reorder the columns
race_age_inc_snap_ownership_us.columns = ['total_pop', 'med_hh_inc', 'hh_snap', 'hh_renter_occupied', 'geography', 'fips', 'year', 'under_age_5', 'age_65_and_over', 'non_white',
                                                'pct_non_white', 'pct_under_age_5', 'pct_age_65_and_over', 'pct_under_age_5_or_age_65_and_over', 'pct_hh_snap', 'pct_hh_renter_occupied']
race_age_inc_snap_ownership_us = race_age_inc_snap_ownership_us[['year', 'fips', 'geography', 'total_pop', 'non_white', 'pct_non_white',
                                                                                     'under_age_5', 'pct_under_age_5', 'age_65_and_over', 'pct_age_65_and_over', 'pct_under_age_5_or_age_65_and_over',
                                                                                     'med_hh_inc', 'hh_snap', 'pct_hh_snap', 'hh_renter_occupied', 'pct_hh_renter_occupied']]

race_age_inc_snap_ownership_us = race_age_inc_snap_ownership_us
race_age_inc_snap_ownership_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 1
Data columns (total 16 columns):
year                                  9 non-null int64
fips                                  9 non-null object
geography                             9 non-null object
total_pop                             9 non-null int64
non_white                             9 non-null int64
pct_non_white                         9 non-null float64
under_age_5                           9 non-null int64
pct_under_age_5                       9 non-null float64
age_65_and_over                       9 non-null int64
pct_age_65_and_over                   9 non-null float64
pct_under_age_5_or_age_65_and_over    9 non-null float64
med_hh_inc                            9 non-null int64
hh_snap                               9 non-null int64
pct_hh_snap                           9 non-null float64
hh_renter_occupied                    9 non-null int64
pct_hh_renter_occupied                9 non-null float64
dtype

## Working Age Population

### Counties and county equivalents

#### Make the API calls

In [12]:
# Iterate through the years
working_age_pop_counties = read_or_save(
    'working_age_pop_counties',
    lambda: get_census_years(2009, 2018, working_age_pop_cols, 'county')
)

#### Format the returned data

In [13]:
# Convert data columns to numeric
working_age_pop_counties[working_age_pop_cols] = working_age_pop_counties[working_age_pop_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
working_age_pop_counties['working_age_pop_counties'] = working_age_pop_counties[working_age_pop_cols[:26]].sum(axis=1)
working_age_pop_counties['fips'] = working_age_pop_counties['state'] + working_age_pop_counties['county']

# Drop the columns we no longer need
working_age_pop_counties = working_age_pop_counties.drop(working_age_pop_cols, axis=1)
working_age_pop_counties = working_age_pop_counties.drop(['state', 'county'], axis=1)

# Rename and reorder the columns
working_age_pop_counties.columns = ['geography', 'year', 'working_age_pop', 'fips']
working_age_pop_counties = working_age_pop_counties[['year', 'fips', 'geography', 'working_age_pop']]

working_age_pop_counties_counties = working_age_pop_counties
working_age_pop_counties_counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28985 entries, 1 to 3220
Data columns (total 4 columns):
year               28985 non-null int64
fips               28985 non-null object
geography          28985 non-null object
working_age_pop    28985 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.1+ MB


### Native American Reservations

#### Make the API calls

In [14]:
# Iterate through the years
working_age_pop_reservations = read_or_save(
    'working_age_pop_reservations',
    lambda: get_census_years(2009, 2018, working_age_pop_cols, 'american indian area/alaska native area/hawaiian home land')
)

#### Format the returned data

In [15]:
# Convert data columns to numeric
working_age_pop_reservations[working_age_pop_cols] = working_age_pop_reservations[working_age_pop_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
working_age_pop_reservations['working_age_pop_reservations'] = working_age_pop_reservations[working_age_pop_cols[:26]].sum(axis=1)

# Drop the columns we no longer need
working_age_pop_reservations = working_age_pop_reservations.drop(working_age_pop_cols, axis=1)

# Rename and reorder the columns
working_age_pop_reservations.columns = ['geography', 'fips', 'year', 'working_age_pop']
working_age_pop_reservations = working_age_pop_reservations[['year', 'fips', 'geography', 'working_age_pop']]

working_age_pop_reservations = working_age_pop_reservations
working_age_pop_reservations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6200 entries, 1 to 694
Data columns (total 4 columns):
year               6200 non-null int64
fips               6200 non-null object
geography          6200 non-null object
working_age_pop    6200 non-null int64
dtypes: int64(2), object(2)
memory usage: 242.2+ KB


### United States

#### Make the API calls

In [16]:
# Iterate through the years
working_age_pop_us = read_or_save(
    'working_age_pop_us',
    lambda: get_census_years(2009, 2018, working_age_pop_cols, 'us')
)

#### Format the returned data

In [17]:
# Convert data columns to numeric
working_age_pop_us[working_age_pop_cols] = working_age_pop_us[working_age_pop_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
working_age_pop_us['working_age_pop_us'] = working_age_pop_us[working_age_pop_cols[:26]].sum(axis=1)

# Drop the columns we no longer need
working_age_pop_us = working_age_pop_us.drop(working_age_pop_cols, axis=1)

# Rename and reorder the columns
working_age_pop_us.columns = ['geography', 'fips', 'year', 'working_age_pop']
working_age_pop_us = working_age_pop_us[['year', 'fips', 'geography', 'working_age_pop']]

working_age_pop_us = working_age_pop_us
working_age_pop_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 1
Data columns (total 4 columns):
year               9 non-null int64
fips               9 non-null object
geography          9 non-null object
working_age_pop    9 non-null int64
dtypes: int64(2), object(2)
memory usage: 360.0+ bytes


## Unemployed Population

### Counties and county equivalents

#### Make the API calls

In [18]:
# Iterate through the years
unemployed_pop_counties = read_or_save(
    'unemployed_pop_counties',
    lambda: get_census_years(2009, 2018, unemployed_pop_cols, 'county')
)

#### Format the returned data

In [19]:
# Convert data columns to numeric
unemployed_pop_counties[unemployed_pop_cols] = unemployed_pop_counties[unemployed_pop_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
unemployed_pop_counties['unemployed_pop_counties'] = unemployed_pop_counties[unemployed_pop_cols[:26]].sum(axis=1)
unemployed_pop_counties['fips'] = unemployed_pop_counties['state'] + unemployed_pop_counties['county']

# Drop the columns we no longer need
unemployed_pop_counties = unemployed_pop_counties.drop(unemployed_pop_cols, axis=1)
unemployed_pop_counties = unemployed_pop_counties.drop(['state', 'county'], axis=1)

# Rename and reorder the columns
unemployed_pop_counties.columns = ['geography', 'year', 'unemployed_pop', 'fips']
unemployed_pop_counties = unemployed_pop_counties[['year', 'fips', 'geography', 'unemployed_pop']]

unemployed_pop_counties = unemployed_pop_counties
unemployed_pop_counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28985 entries, 1 to 3220
Data columns (total 4 columns):
year              28985 non-null int64
fips              28985 non-null object
geography         28985 non-null object
unemployed_pop    28985 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.1+ MB


### Native American Reservations

#### Make the API calls

In [20]:
# Iterate through the years
unemployed_pop_reservations = read_or_save(
    'unemployed_pop_reservations',
    lambda: get_census_years(2009, 2018, unemployed_pop_cols, 'american indian area/alaska native area/hawaiian home land')
)

#### Format the returned data

In [21]:
# Convert data columns to numeric
unemployed_pop_reservations[unemployed_pop_cols] = unemployed_pop_reservations[unemployed_pop_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
unemployed_pop_reservations['unemployed_pop_reservations'] = unemployed_pop_reservations[unemployed_pop_cols[:26]].sum(axis=1)

# Drop the columns we no longer need
unemployed_pop_reservations = unemployed_pop_reservations.drop(unemployed_pop_cols, axis=1)

# Rename and reorder the columns
unemployed_pop_reservations.columns = ['geography', 'fips', 'year', 'unemployed_pop',]
unemployed_pop_reservations = unemployed_pop_reservations[['year', 'fips', 'geography', 'unemployed_pop']]

unemployed_pop_reservations = unemployed_pop_reservations
unemployed_pop_reservations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6200 entries, 1 to 694
Data columns (total 4 columns):
year              6200 non-null int64
fips              6200 non-null object
geography         6200 non-null object
unemployed_pop    6200 non-null int64
dtypes: int64(2), object(2)
memory usage: 242.2+ KB


### United States

#### Make the API calls

In [22]:
# Iterate through the years
unemployed_pop_us = read_or_save(
    'unemployed_pop_us',
    lambda: get_census_years(2009, 2018, unemployed_pop_cols, 'us')
)

#### Format the returned data

In [23]:
# Convert data columns to numeric
unemployed_pop_us[unemployed_pop_cols] = unemployed_pop_us[unemployed_pop_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Create new columns
unemployed_pop_us['unemployed_pop_us'] = unemployed_pop_us[unemployed_pop_cols[:26]].sum(axis=1)

# Drop the columns we no longer need
unemployed_pop_us = unemployed_pop_us.drop(unemployed_pop_cols, axis=1)

# Rename and reorder the columns
unemployed_pop_us.columns = ['geography', 'fips', 'year', 'unemployed_pop',]
unemployed_pop_us = unemployed_pop_us[['year', 'fips', 'geography', 'unemployed_pop']]

unemployed_pop_us = unemployed_pop_us
unemployed_pop_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 1
Data columns (total 4 columns):
year              9 non-null int64
fips              9 non-null object
geography         9 non-null object
unemployed_pop    9 non-null int64
dtypes: int64(2), object(2)
memory usage: 360.0+ bytes


## Join the county dataframes

In [24]:
county_dfs = [race_age_inc_snap_ownership_counties, working_age_pop_counties, unemployed_pop_counties]
counties = reduce(lambda left, right: pd.merge(left, right, on=['year', 'fips', 'geography']), county_dfs)
counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28985 entries, 0 to 28984
Data columns (total 18 columns):
year                                  28985 non-null int64
fips                                  28985 non-null object
geography                             28985 non-null object
total_pop                             28985 non-null float64
non_white                             28985 non-null float64
pct_non_white                         28985 non-null float64
under_age_5                           28985 non-null float64
pct_under_age_5                       28985 non-null float64
age_65_and_over                       28985 non-null float64
pct_age_65_and_over                   28985 non-null float64
pct_under_age_5_or_age_65_and_over    28985 non-null float64
med_hh_inc                            28984 non-null float64
hh_snap                               28985 non-null float64
pct_hh_snap                           28985 non-null float64
hh_renter_occupied                    289

## Join the reservations dataframes

In [25]:
reservation_dfs = [race_age_inc_snap_ownership_reservations, working_age_pop_reservations, unemployed_pop_reservations]
reservations = reduce(lambda left, right: pd.merge(left, right, on=['year', 'fips', 'geography']), reservation_dfs)
reservations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6200 entries, 0 to 6199
Data columns (total 18 columns):
year                                  6200 non-null int64
fips                                  6200 non-null object
geography                             6200 non-null object
total_pop                             6200 non-null float64
non_white                             6200 non-null float64
pct_non_white                         5474 non-null float64
under_age_5                           6200 non-null float64
pct_under_age_5                       5474 non-null float64
age_65_and_over                       6200 non-null float64
pct_age_65_and_over                   5474 non-null float64
pct_under_age_5_or_age_65_and_over    5474 non-null float64
med_hh_inc                            6056 non-null float64
hh_snap                               6200 non-null float64
pct_hh_snap                           5454 non-null float64
hh_renter_occupied                    6200 non-null float

## Join the United States dataframes

In [26]:
us_dfs = [race_age_inc_snap_ownership_us, working_age_pop_us, unemployed_pop_us]
us = reduce(lambda left, right: pd.merge(left, right, on=['year', 'fips', 'geography']), us_dfs)
us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 0 to 8
Data columns (total 18 columns):
year                                  9 non-null int64
fips                                  9 non-null object
geography                             9 non-null object
total_pop                             9 non-null int64
non_white                             9 non-null int64
pct_non_white                         9 non-null float64
under_age_5                           9 non-null int64
pct_under_age_5                       9 non-null float64
age_65_and_over                       9 non-null int64
pct_age_65_and_over                   9 non-null float64
pct_under_age_5_or_age_65_and_over    9 non-null float64
med_hh_inc                            9 non-null int64
hh_snap                               9 non-null int64
pct_hh_snap                           9 non-null float64
hh_renter_occupied                    9 non-null int64
pct_hh_renter_occupied                9 non-null float64
worki

## Concatenate the county and reservations dataframes

In [27]:
census = pd.concat([counties, reservations])
census.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35185 entries, 0 to 6199
Data columns (total 18 columns):
year                                  35185 non-null int64
fips                                  35185 non-null object
geography                             35185 non-null object
total_pop                             35185 non-null float64
non_white                             35185 non-null float64
pct_non_white                         34459 non-null float64
under_age_5                           35185 non-null float64
pct_under_age_5                       34459 non-null float64
age_65_and_over                       35185 non-null float64
pct_age_65_and_over                   34459 non-null float64
pct_under_age_5_or_age_65_and_over    34459 non-null float64
med_hh_inc                            35040 non-null float64
hh_snap                               35185 non-null float64
pct_hh_snap                           34439 non-null float64
hh_renter_occupied                    3518

## Join the resulting dataframe with the United States dataframe

In [28]:
census = census.merge(us, on=['year'], suffixes=["", "_us"])
census.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35185 entries, 0 to 35184
Data columns (total 35 columns):
year                                     35185 non-null int64
fips                                     35185 non-null object
geography                                35185 non-null object
total_pop                                35185 non-null float64
non_white                                35185 non-null float64
pct_non_white                            34459 non-null float64
under_age_5                              35185 non-null float64
pct_under_age_5                          34459 non-null float64
age_65_and_over                          35185 non-null float64
pct_age_65_and_over                      34459 non-null float64
pct_under_age_5_or_age_65_and_over       34459 non-null float64
med_hh_inc                               35040 non-null float64
hh_snap                                  35185 non-null float64
pct_hh_snap                              34439 non-null float64

## Calculate the unemployment rate

In [29]:
# Create new columns
census['unemployment_rate'] = census['unemployed_pop'] / census['working_age_pop']
census['unemployment_rate_us'] = census['unemployed_pop_us'] / census['working_age_pop_us']

# Drop the columns we no longer need
census = census.drop(['working_age_pop', 'working_age_pop_us'], axis=1)

# Reorder the columns
census = census[['year', 'fips', 'geography', 'total_pop', 'non_white', 'pct_non_white', 'under_age_5', 'pct_under_age_5', 'age_65_and_over', 'pct_age_65_and_over',
                 'pct_under_age_5_or_age_65_and_over', 'unemployed_pop', 'unemployment_rate', 'med_hh_inc', 'hh_snap', 'pct_hh_snap', 'hh_renter_occupied',
                 'pct_hh_renter_occupied', 'fips_us', 'geography_us', 'total_pop_us', 'non_white_us', 'pct_non_white_us', 'under_age_5_us', 'pct_under_age_5_us',
                 'age_65_and_over_us', 'pct_age_65_and_over_us', 'pct_under_age_5_or_age_65_and_over_us', 'med_hh_inc_us', 'hh_snap_us', 'pct_hh_snap_us',
                 'hh_renter_occupied_us', 'pct_hh_renter_occupied_us', 'unemployed_pop_us']]
census.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35185 entries, 0 to 35184
Data columns (total 34 columns):
year                                     35185 non-null int64
fips                                     35185 non-null object
geography                                35185 non-null object
total_pop                                35185 non-null float64
non_white                                35185 non-null float64
pct_non_white                            34459 non-null float64
under_age_5                              35185 non-null float64
pct_under_age_5                          34459 non-null float64
age_65_and_over                          35185 non-null float64
pct_age_65_and_over                      34459 non-null float64
pct_under_age_5_or_age_65_and_over       34459 non-null float64
unemployed_pop                           35185 non-null int64
unemployment_rate                        34389 non-null float64
med_hh_inc                               35040 non-null float64
h

## Export the data to csv

In [30]:
census.to_csv('data/census.csv', index=False)